In [1]:
import json
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from xgboost import XGBClassifier
import pandas as pd

label_encoder = LabelEncoder()
data = {
 "id": "0x5fe9411b9d5e1e4ae2a2c59deb9c87e6db04613c",
 "Label": None,
 "mint_count_per_week": 0.9143520618276156,
 "burn_count_per_week": 0.9143520618276156,
 "mint_ratio": 0.07142857142857142,
 "swap_ratio": 0.8571428571428571,
 "burn_ratio": 0.07142857142857142,
 "mint_mean_period": 0.0,
 "swap_mean_period": 0.11348231306926498,
 "burn_mean_period": 1.0,
 "swap_in_per_week": 10.972224741931388,
 "swap_out_per_week": 0.0,
 "swap_rate": 12.0,
 "lp_avg": 100.0,
 "lp_std": 0.0,
 "lp_creator_holding_ratio": 0,
 "lp_lock_ratio": 0,
 "token_burn_ratio": 100,
 "token_creator_holding_ratio": 68.09
}

In [2]:
# read dataframe
label_encoder = LabelEncoder()
dataframe = pd.read_csv('./../Dataset/type_7days/Dataset_v1.9.csv')
dataframe.dropna(inplace=True) 
dataframe.rename(columns=lambda x : x.lower(), inplace=True)

# label encoder
dataframe['label'] = label_encoder.fit_transform(dataframe['label'])

# head
dataframe.head()

,id,label,mint_count_per_week,burn_count_per_week,mint_ratio,swap_ratio,burn_ratio,mint_mean_period,swap_mean_period,burn_mean_period,swap_in_per_week,swap_out_per_week,swap_rate,lp_avg,lp_std,lp_creator_holding_ratio,lp_lock_ratio,token_burn_ratio,token_creator_holding_ratio,number_of_token_creation_of_creator
0,0x3cd1c0b98be4451ca51265bbaeb76cf7b31e1c02,0,531.195902,39.839693,0.481928,0.481928,0.036145,0.393718,0.032614,0.507770,304.375252,226.820650,1.338785,0.119760,0.126458,0.000000e+00,0.000000,0.000000e+00,1.302617e-01,1
1,0x0c52de7bb69edd144d58d772fab1cd196919f5ef,0,54.542476,2.001559,0.377163,0.608997,0.013841,0.805150,0.640197,0.551141,49.038189,39.030396,1.240506,0.332226,0.431320,0.000000e+00,0.000000,4.000000e-07,1.000000e-08,1
2,0xa00d47b4b304792eb07b09233467b690db847c91,0,230.772631,110.130106,0.274242,0.594884,0.130874,0.452930,0.053773,0.602639,240.784458,259.806931,0.925000,0.377358,4.649864,0.000000e+00,0.064726,0.000000e+00,0.000000e+00,1
3,0x114e40ba90e9d8b002bc6936e3299daa393017bb,0,58.779514,8.540613,0.103175,0.881834,0.014991,0.759745,0.190998,0.711122,318.012241,184.376765,1.720109,0.467290,4.754048,1.480000e-08,0.693921,3.748588e-03,0.000000e+00,1
4,0x40829a59080a12f16bb8fba22354a6a42c810aab,0,94.726400,12.028749,0.155812,0.824402,0.019786,0.508112,0.002882,0.630434,313.248679,187.949207,1.662234,0.476190,5.466877,0.000000e+00,0.790202,0.000000e+00,1.000000e-08,1


In [3]:
# preprocessing
x = dataframe.drop(columns=['id', 'label', 'number_of_token_creation_of_creator'], axis=1)
y = dataframe['label']
scaler = StandardScaler().fit(x)
x = pd.DataFrame(scaler.transform(x))
x.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,29.114768,16.199214,5.453155,-3.622846,-0.212956,3.455632,-0.928498,-0.486208,2.555793,3.097065,-0.225893,-5.734484,-0.227774,-1.981697,-0.476000,-0.147103,-0.032288
1,2.862023,0.434613,4.090425,-2.494949,-0.641532,7.378519,2.663246,-0.383230,-0.031837,0.175193,-0.228837,-5.721746,-0.196021,-1.981697,-0.476000,-0.147102,-0.033027
2,12.568290,45.484494,2.751680,-2.620216,1.607322,4.020207,-0.803415,-0.260955,1.911353,3.610306,-0.238287,-5.719040,0.243358,-1.981697,-0.305780,-0.147103,-0.033027
3,3.095387,3.158997,0.526531,-0.073168,-0.619428,6.945604,0.007796,-0.003378,2.693993,2.436671,-0.214472,-5.713648,0.254209,-1.981697,1.348896,-0.143794,-0.033027
4,5.075242,4.612268,1.211211,-0.582949,-0.527300,4.546346,-1.104255,-0.194959,2.645718,2.492255,-0.216206,-5.713115,0.328454,-1.981697,1.602098,-0.147103,-0.033027


In [4]:
# Training model
xgboost_model = XGBClassifier(objective="binary:logistic", n_estimators=20, random_state=42)
xgboost_model.fit(x, y)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=20, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [5]:
# Convert data record to DataFrame
data_df = pd.DataFrame([data])  # Chuyển đổi dữ liệu bản ghi thành DataFrame
print(data_df.shape)
type(data_df)
data_df

(1, 19)


,id,Label,mint_count_per_week,burn_count_per_week,mint_ratio,swap_ratio,burn_ratio,mint_mean_period,swap_mean_period,burn_mean_period,swap_in_per_week,swap_out_per_week,swap_rate,lp_avg,lp_std,lp_creator_holding_ratio,lp_lock_ratio,token_burn_ratio,token_creator_holding_ratio
0,0x5fe9411b9d5e1e4ae2a2c59deb9c87e6db04613c,None,0.914352,0.914352,0.071429,0.857143,0.071429,0.0,0.113482,1.0,10.972225,0.0,12.0,100.0,0.0,0,0,100,68.09


In [6]:

# Preprocess and predict data
data_df.rename(columns=lambda x: x.lower(), inplace=True)
data_df = data_df.drop(columns=['id', 'label'], axis=1)
data_df = pd.DataFrame(scaler.transform(data_df))
type(data_df)
data_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,-0.091665,-0.018352,0.113595,-0.292335,0.465042,-0.298357,-0.450441,0.682525,-0.417604,-0.432089,0.093423,0.253702,-0.240945,-1.981697,-0.476,88.109669,0.35308


In [7]:
# Predict probabilities
probabilities = xgboost_model.predict_proba(data_df)

# Print probabilities of being 0 and 1
print("Probability of being 0:", probabilities[0][0])
print("Probability of being 1:", probabilities[0][1])

Probability of being 0: 0.01655811
Probability of being 1: 0.9834419
